In [1]:
import pandas as pd
import numpy as np
import os
import datetime
folderpath = '/home/jubauser1/BiglotsCode/outputs/'
lastweeksdate = '2017-09-09'

In [2]:
newsalespath = '/home/jubauser1/BigLots/MediaStorm Data Extract week ending 2017-09-16/MediaStormSalesWeekly.txt'
newtrafficpath = '/home/jubauser1/BigLots/MediaStorm Data Extract week ending 2017-09-16/MediaStormTrafficWeekly.txt'
newinventorypath = '/home/jubauser1/BigLots/MediaStorm Data Extract week ending 2017-09-16/MediaStormInventoryWeekly.txt'

In [3]:
closed_onlinestorelist = ['6990','280', '298', '388', '507', '824', '1098', 
                          '1120', '1148', '1182', '1374', '1773', '1822', '1913',
                          '1967', '4085', '4099', '4113', '145',
                          '4165', '4280', '4326', '4362', '4382', '4428', '5133']

In [4]:
dfsales = pd.read_csv(folderpath + 'combinedsales'+ lastweeksdate + '.csv',sep = '|',dtype = 'str')
df = pd.read_csv(newsalespath,sep = '|',dtype = 'str')
a = df.columns
print("new sales data column header matches:")
print(a == ['location_id', 'week_end_dt', 'fiscal_week_nbr', 'gross_sales_amt',
       'gross_transaction_cnt', 'class_code_id', 'class_gross_sales_amt'])
dfsales = dfsales.append(df,ignore_index = True)
a = (len(dfsales.index))
dfsales = dfsales.drop_duplicates(['location_id', 'week_end_dt', 'fiscal_week_nbr', 
       'class_code_id'])
b = (len(dfsales.index))
if a==b:
    print("")
else:
    print("last week traffic data duplication deduped")

new sales data column header matches:
[ True  True  True  True  True  True  True]



In [5]:
recentweek = (max(dfsales['week_end_dt']))

In [6]:
dfsales.to_csv(folderpath + 'combinedsales'+ recentweek + '.csv',index = False,sep = '|')

In [6]:
dfsales = dfsales[~dfsales['location_id'].isin(closed_onlinestorelist)]

In [7]:
outputpath = folderpath +'Output_' + recentweek +'/'
try:
    os.stat(outputpath)
except:
    os.mkdir(outputpath)

In [8]:
dfnodata = dfsales[(dfsales['class_gross_sales_amt'] == '?')&\
                   (dfsales['week_end_dt'] == recentweek)]
dfnodata.to_csv(outputpath + 'sales_nodata.csv',index = False)
print("stores with ? sales/transaction: " + str(len(dfnodata.index)))

stores with ? sales/transaction: 0


In [9]:
dfsales['week_end_dt'] = pd.to_datetime(dfsales['week_end_dt'])
dfsales = dfsales[dfsales['class_gross_sales_amt']!='?']
dfsales = dfsales.reset_index(drop = True)

dfsales['gross_sales_amt'] = dfsales['gross_sales_amt'].astype('float')
dfsales['gross_transaction_cnt'] = dfsales['gross_transaction_cnt'].astype('float')
dfsales['class_gross_sales_amt'] = dfsales['class_gross_sales_amt'].astype('float')

In [10]:
dfweeklist = dfsales[['week_end_dt','fiscal_week_nbr']].drop_duplicates()
dfweeklist = dfweeklist.sort_values('week_end_dt',ascending = False)
dfweeklist.reset_index(drop = True,inplace = True)
dfweeklist.reset_index(inplace = True)

dfweeklist_wow = dfweeklist.copy()
dfweeklist_wow['index'] = dfweeklist_wow['index'] - 1
dfweeklist_wow = dfweeklist_wow[['index','week_end_dt']]
dfweeklist_wow.columns = ['index','weeklastweek']

dfweeklist = dfweeklist[dfweeklist['index']<104]
dfweeklist.reset_index(drop = True,inplace = True)
dfweeklist['year'] = np.ceil((dfweeklist['index'] + 1)/52)

dfweeklist1 = dfweeklist[dfweeklist['year'] == 1]
dfweeklist1 = dfweeklist1[['index', 'week_end_dt', 'fiscal_week_nbr']]
dfweeklist2 = dfweeklist[dfweeklist['year'] == 2]
dfweeklist2 = dfweeklist2[['week_end_dt', 'fiscal_week_nbr']]
dfweeklist2.columns = ['weeklastyear', 'fiscal_week_nbr']

dfweeklist = pd.merge(dfweeklist1,dfweeklist2,on ='fiscal_week_nbr' )
dfweeklist = pd.merge(dfweeklist,dfweeklist_wow,on ='index')
del dfweeklist1,dfweeklist2,dfweeklist_wow

dfweeklist.to_csv(outputpath + 'weeklist.csv',index = False)

In [11]:
recentweek_date = (max(dfsales['week_end_dt']))
dfcheck = dfsales[dfsales['week_end_dt'] == recentweek_date]

In [12]:
dfcheck_total1 = dfcheck[['location_id', 'week_end_dt', 'fiscal_week_nbr',
                         'gross_sales_amt','gross_transaction_cnt']].drop_duplicates()
a = (len(dfcheck_total1.index))
dfcheck_total1 = dfcheck_total1.drop_duplicates(['location_id', 'week_end_dt', 'fiscal_week_nbr'])
b = (len(dfcheck_total1.index))
if a==b:
    print("")
else:
    print("last week sales multiple gross sales/trasaction in the same store")

dfcheck_total2 = dfcheck[['location_id', 'week_end_dt', 'fiscal_week_nbr',
                         'class_gross_sales_amt']].groupby(['location_id', 'week_end_dt', 'fiscal_week_nbr']).sum()
dfcheck_total2.reset_index(inplace = True)

dfcheck_total = pd.merge(dfcheck_total1,dfcheck_total2,
                        on = ['location_id', 'week_end_dt', 'fiscal_week_nbr'] ,
                        how = 'outer')
del dfcheck_total1,dfcheck_total2

dfcheck_zero = dfcheck_total[(dfcheck_total['class_gross_sales_amt']<=0)|\
                            (dfcheck_total['gross_transaction_cnt']<=0) ]

dfcheck_zero.to_csv(outputpath + 'zerosales.csv',index = False)
print("stores with zero sales/transaction: " + str(len(dfcheck_zero.index)))
del dfcheck_zero

dfcheck_total['TotalDiff'] = dfcheck_total['gross_sales_amt']-dfcheck_total['class_gross_sales_amt']
dfcheck_total['TotalDiff'] = dfcheck_total['TotalDiff'].round()
dfcheck_totalnonmatch = dfcheck_total[dfcheck_total['TotalDiff']!=0]
print("stores gross sales can not match sum of class sales: " + str(len(dfcheck_totalnonmatch.index)))

dfcheck_totalnonmatch.to_csv(outputpath + 'totalnonmatch.csv',index = False)
del dfcheck_totalnonmatch

dfcheck_zeroclass = dfcheck[(dfcheck['class_gross_sales_amt']==0)]
dfcheck_zeroclass.to_csv(outputpath + 'zeroclasssales.csv',index = False)
print("stores with zero class sales: " + str(len(dfcheck_zeroclass.index)))
del dfcheck_zeroclass
del dfcheck


stores with zero sales/transaction: 0
stores gross sales can not match sum of class sales: 1
stores with zero class sales: 0


In [13]:
dfsales_total1 = dfsales[['location_id', 'week_end_dt', 'fiscal_week_nbr',
                         'gross_sales_amt','gross_transaction_cnt']].drop_duplicates()
dfsales_total1 = dfsales_total1.drop_duplicates(['location_id', 'week_end_dt', 'fiscal_week_nbr'])

dfsales_total2 = dfsales[['location_id', 'week_end_dt', 'fiscal_week_nbr',
                         'class_gross_sales_amt']].groupby(['location_id', 'week_end_dt', 'fiscal_week_nbr']).sum()
dfsales_total2.reset_index(inplace = True)

dfsales_total = pd.merge(dfsales_total1,dfsales_total2,
                        on = ['location_id', 'week_end_dt', 'fiscal_week_nbr'] ,
                        how = 'outer')
del dfsales_total1,dfsales_total2

In [14]:
dfstore = pd.read_table('/home/jubauser1/BigLots/static_files/MediaStormStoresList_0913.txt',
                        sep = '|',dtype = 'str')
dfstore['open_dt'] = pd.to_datetime(dfstore['open_dt'])
dfstore['open_dtwd'] = dfstore['open_dt'].dt.dayofweek
dfstore['open_wk'] = np.where(dfstore['open_dtwd']<=5,
                       dfstore['open_dt'].apply(lambda x:x+datetime.timedelta(days=(5-x.weekday()))),
                       dfstore['open_dt'].apply(lambda x:x+datetime.timedelta(days=(12-x.weekday()))))

In [15]:
dfstorematch = dfsales_total[['location_id']].drop_duplicates()
dfstorematch = pd.merge(dfstorematch,dfstore[['location_id','address_line_1']],
                        on = 'location_id',how = 'left')
dfstorematch['address_line_1'].fillna('empty',inplace = True)
dfstorematch = dfstorematch[dfstorematch['address_line_1']=='empty']
print("stores w/o detailed info: ")
print(dfstorematch['location_id'].unique())
del dfstorematch

stores w/o detailed info: 
[]


In [16]:
dfsales_total = pd.merge(dfsales_total,dfstore[['location_id','open_wk']],
                        on = 'location_id',how = 'left')
dfsales_total['open_wk'].fillna(datetime.datetime.strptime(str(20200101), '%Y%m%d').date(),inplace = True)

In [17]:
dftraffic = pd.read_csv(folderpath + 'combinedtraffic'+ lastweeksdate + '.csv',
               sep = '|',dtype = 'str')

df = pd.read_csv(newtrafficpath,sep = '|',dtype = 'str')
a = df.columns
print("new traffic data column header matches:")
print(a == ['location_id', 'week_end_dt', 'fiscal_week_nbr', 'traffic_day_1',
       'traffic_day_2', 'traffic_day_3', 'traffic_day_4', 'traffic_day_5',
       'traffic_day_6', 'traffic_day_7'])
dftraffic = dftraffic.append(df,ignore_index = True)
a = (len(dftraffic.index))
dftraffic = dftraffic.drop_duplicates(['location_id', 'week_end_dt', 'fiscal_week_nbr'])
b = (len(dftraffic.index))
if a==b:
    print("")
else:
    print("last week traffic data duplication deduped")
dftraffic.to_csv(folderpath + 'combinedtraffic'+ recentweek + '.csv',index = False,sep = '|')

dftraffic['traffic_week'] = 0 
for i in ['traffic_day_1','traffic_day_2', 'traffic_day_3', 'traffic_day_4',
          'traffic_day_5', 'traffic_day_6', 'traffic_day_7']:
    dftraffic[i] = dftraffic[i].astype('float')
    dftraffic['traffic_week'] = dftraffic['traffic_week'] +dftraffic[i]
dftraffic['week_end_dt'] = pd.to_datetime(dftraffic['week_end_dt'])

new traffic data column header matches:
[ True  True  True  True  True  True  True  True  True  True]



In [18]:
dfinventory = pd.read_csv(folderpath + 'combinedinventory'+ lastweeksdate + '.csv',
               sep = '|',dtype = 'str')

df = pd.read_csv(newinventorypath,sep = '|',dtype = 'str')
a = df.columns
print("new inventory data column header matches:")
print(a == ['location_id', 'week_end_dt', 'fiscal_week_nbr', 'class_code_id','on_hand'])
df.columns = ['location_id', 'week_end_dt', 'fiscal_week_nbr', 'class_code_id','on_hand']
dfinventory = dfinventory.append(df,ignore_index = True)
a = (len(dfinventory.index))
dfinventory = dfinventory.drop_duplicates(['location_id', 'week_end_dt', 'fiscal_week_nbr', 'class_code_id'])
b = (len(dfinventory.index))
if a==b:
    print("")
else:
    print("last week inventory data duplication deduped")

new inventory data column header matches:
[ True  True  True  True False]



In [20]:
dfinventory.to_csv(folderpath + 'combinedinventory'+ recentweek + '.csv',index = False,sep = '|')

In [19]:
dfinventory['week_end_dt'] = pd.to_datetime(dfinventory['week_end_dt'])
dfinventory['on_hand'] = dfinventory['on_hand'].astype('float')
dfinventory_total = dfinventory.groupby(['location_id', 'week_end_dt', 'fiscal_week_nbr']).sum()
dfinventory_total.reset_index(inplace = True)

In [20]:
dfsales_total_recent = pd.merge(dfsales_total,
                                dfweeklist[['week_end_dt', 'fiscal_week_nbr','weeklastyear','weeklastweek']],
                                on= ['week_end_dt', 'fiscal_week_nbr'])

dfsales_total_lastyear = pd.merge(dfsales_total,
                                 dfweeklist[['weeklastyear']],
                                 left_on= 'week_end_dt',right_on = 'weeklastyear')

dfsales_total_lastyear = dfsales_total_lastyear[['location_id','gross_transaction_cnt', 'class_gross_sales_amt','weeklastyear']]
dfsales_total_lastyear.columns = ['location_id','gross_transaction_cnt_ly', 'class_gross_sales_amt_ly','weeklastyear']

dfsales_total_recent = pd.merge(dfsales_total_recent,dfsales_total_lastyear,
                                on = ['location_id','weeklastyear'],how = 'outer')
dfsales_total_recent.fillna(0,inplace = True)

dfsales_total_recent['Store_Category'] = np.where(dfsales_total_recent['open_wk']>=dfsales_total_recent['weeklastyear'],'New',
                                         np.where((dfsales_total_recent['gross_transaction_cnt_ly']==0)&(dfsales_total_recent['class_gross_sales_amt_ly']==0),
                                         'Converted',
                                         np.where((dfsales_total_recent['gross_transaction_cnt']==0)&(dfsales_total_recent['class_gross_sales_amt']==0),
                                        'Converted','Complete')))

dfsales_total_lastweek = pd.merge(dfsales_total,
                                 dfweeklist[['weeklastweek']],
                                 left_on= 'week_end_dt',right_on = 'weeklastweek')

dfsales_total_lastweek = dfsales_total_lastweek[['location_id','gross_transaction_cnt', 'class_gross_sales_amt','weeklastweek']]
dfsales_total_lastweek.columns = ['location_id','gross_transaction_cnt_lw', 'class_gross_sales_amt_lw','weeklastweek']
dfsales_total_recent = pd.merge(dfsales_total_recent,dfsales_total_lastweek,
                                on = ['location_id','weeklastweek'],how = 'left')
dfsales_total_recent.fillna(0,inplace = True)

dfsales_total_recent['week_end_dt'] = np.where(dfsales_total_recent['week_end_dt']=='1970-01-01',
                                       dfsales_total_recent['weeklastyear'] + pd.DateOffset(364),
                                       dfsales_total_recent['week_end_dt'])
dfsales_total_recent['weeklastyear'] = np.where(dfsales_total_recent['weeklastyear']=='1970-01-01',
                                       dfsales_total_recent['week_end_dt'] + pd.DateOffset(-364),
                                       dfsales_total_recent['weeklastyear'])

In [28]:
dfallstorelist = dfstore[~dfstore['location_id'].isin(closed_onlinestorelist)]
dfallstorelist.reset_index(drop = True, inplace = True)
dfweeklist2 = dfweeklist.copy()
dfallstorelist['concat'] = 1
dfweeklist2['concat'] = 1
dfallstorelist = pd.merge(dfallstorelist,dfweeklist2,on='concat')
dfallstorelist = dfallstorelist[['location_id','week_end_dt']]
dfallstorelist = pd.merge(dfallstorelist,dfsales_total_recent,on=['location_id','week_end_dt'],
                         how = 'left')

dfallstorelist.fillna(0,inplace = True)
dfallstorelist = dfallstorelist[(dfallstorelist['gross_sales_amt']==0)&\
                               (dfallstorelist['gross_transaction_cnt']==0)&\
                               (dfallstorelist['class_gross_sales_amt']==0)&\
                               (dfallstorelist['gross_transaction_cnt_ly']==0)&\
                               (dfallstorelist['class_gross_sales_amt_ly']==0)]
dfallstorelist = dfallstorelist.sort_values(['week_end_dt','location_id'],ascending = [0,1])
dfallstorelist = dfallstorelist[['week_end_dt','location_id']]
dfallstorelist.to_csv(outputpath + 'nobothyeardatastores.csv',index = False)
print("stores with no 2017&2016 sales and transaction data: " + str(len(dfallstorelist.index)))
test = dfallstorelist[dfallstorelist['week_end_dt']==recentweek_date]
print("Last Week: stores with no 2017&2016 sales and transaction data: " + str(len(test.index)))
del test,dfweeklist2

stores with no 2017&2016 sales and transaction data: 466
Last Week: stores with no 2017&2016 sales and transaction data: 0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [25]:
dfallstorelist

,week_end_dt,location_id
65262,2017-09-02,5350
65263,2017-08-26,5350
74207,2017-08-26,5351
65264,2017-08-19,5350
74208,2017-08-19,5351
74157,2017-08-12,5348
65265,2017-08-12,5350
74209,2017-08-12,5351
74158,2017-08-05,5348
65266,2017-08-05,5350


In [26]:
dfallstorelist.shape

(466, 2)

In [27]:
len(dfallstorelist['location_id'].unique())

16

In [40]:
dfsales_total_recent = pd.merge(dfsales_total_recent,dftraffic,
                                on=['location_id', 'week_end_dt', 'fiscal_week_nbr'],how = 'left')

dftraffic2 = dftraffic[['location_id', 'week_end_dt','traffic_day_1',
       'traffic_day_2', 'traffic_day_3', 'traffic_day_4', 'traffic_day_5',
       'traffic_day_6', 'traffic_day_7','traffic_week']]
dftraffic2.columns = ['location_id', 'weeklastyear','traffic_day_1_ly',
       'traffic_day_2_ly', 'traffic_day_3_ly', 'traffic_day_4_ly', 'traffic_day_5_ly',
       'traffic_day_6_ly', 'traffic_day_7_ly','traffic_week_ly']

dfsales_total_recent = pd.merge(dfsales_total_recent,dftraffic2,
                                on=['location_id', 'weeklastyear'],how = 'left')
del dftraffic2

In [41]:
dfsales_total_recent = pd.merge(dfsales_total_recent,dfinventory_total,
                                on=['location_id', 'week_end_dt', 'fiscal_week_nbr'],how = 'left')

dfinventory_total2 = dfinventory_total[['location_id', 'week_end_dt','on_hand']]
dfinventory_total2.columns = ['location_id', 'weeklastyear','on_hand_ly']

dfsales_total_recent = pd.merge(dfsales_total_recent,dfinventory_total2,
                                on=['location_id', 'weeklastyear'],how = 'left')
del dfinventory_total2

In [42]:
recentweek_last=datetime.datetime.strptime(recentweek, '%Y-%m-%d').date()
recentweek_last=recentweek_last+datetime.timedelta(days=(-84))

In [43]:
dfsales_total_recent['yoysales'] = dfsales_total_recent['class_gross_sales_amt']/dfsales_total_recent['class_gross_sales_amt_ly'] - 1
dfsales_total_recent['yoytrans'] = dfsales_total_recent['gross_transaction_cnt']/dfsales_total_recent['gross_transaction_cnt_ly'] - 1
dfsales_total_recent['wowsales'] = dfsales_total_recent['class_gross_sales_amt']/dfsales_total_recent['class_gross_sales_amt_lw'] - 1
dfsales_total_recent['wowtrans'] = dfsales_total_recent['gross_transaction_cnt']/dfsales_total_recent['gross_transaction_cnt_lw'] - 1

In [44]:
dfsales_total_recent_delete = dfsales_total_recent[(dfsales_total_recent['Store_Category']=='Complete')&\
                                                   ((abs(dfsales_total_recent['yoysales'])>0.2)&\
                                                   (abs(dfsales_total_recent['yoytrans'])>0.2))]#|\
                                                   #(abs(dfsales_total_recent['wowsales'])>0.2)|\
                                                   #(abs(dfsales_total_recent['wowtrans'])>0.2))]
dfsales_total_recent_delete = dfsales_total_recent_delete.sort_values(['week_end_dt','location_id'],
                                                                     ascending = [0,1])
dfsales_total_recent_delete.to_csv(outputpath + 'highyoy_wowchangestores.csv',index = False)
print("stores with high yoy change: " + str(len(dfsales_total_recent_delete.index)))
test = dfsales_total_recent_delete[dfsales_total_recent_delete['week_end_dt']==recentweek_date]
print("Last Week: stores with high yoy change: " + str(len(test.index)))
del test

stores with high yoy change: 0
Last Week: stores with high yoy change: 0


In [45]:
#dfsales_total_recent =  dfsales_total_recent[(dfsales_total_recent['gross_transaction_cnt']!=0)|\
#                                             (dfsales_total_recent['class_gross_sales_amt']!=0)]
dfsales_total_recent = dfsales_total_recent[(dfsales_total_recent['Store_Category']!='Complete')|\
                                            ((dfsales_total_recent['Store_Category']=='Complete')&\
                                            (abs(dfsales_total_recent['yoysales'])<=0.2)|\
                                            (abs(dfsales_total_recent['yoytrans'])<=0.2))]#&\
                                            #(abs(dfsales_total_recent['wowsales'])<=0.2)&\
                                            #(abs(dfsales_total_recent['wowtrans'])<=0.2))]

In [46]:
dfweeklist2 = dfweeklist[['week_end_dt']]
dfweeklist2['week_end_dt_8w'] = dfweeklist2['week_end_dt']+pd.DateOffset(-84)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
dfweeklist_12plus = dfsales[['week_end_dt']].drop_duplicates()
dfweeklist_12plus = dfweeklist_12plus.sort_values('week_end_dt',ascending = False)
dfweeklist_12plus.reset_index(drop = True,inplace = True)
dfweeklist_12plus.reset_index(inplace = True)
dfweeklist_12plus = dfweeklist_12plus[dfweeklist_12plus['index']<64]
dfweeklist_12plus['weeklastyear'] = dfweeklist_12plus['week_end_dt'] + pd.DateOffset(-364)

In [48]:
dfsales_12plus = pd.merge(dfsales_total,dfweeklist_12plus,on= ['week_end_dt'])

dfsales_12plus_lastyear = pd.merge(dfsales_total,
                                 dfweeklist_12plus[['weeklastyear']],
                                 left_on= 'week_end_dt',right_on = 'weeklastyear')

dfsales_12plus_lastyear = dfsales_12plus_lastyear[['location_id','gross_transaction_cnt', 'class_gross_sales_amt','weeklastyear']]
dfsales_12plus_lastyear.columns = ['location_id','gross_transaction_cnt_ly', 'class_gross_sales_amt_ly','weeklastyear']

dfsales_12plus = pd.merge(dfsales_12plus,dfsales_12plus_lastyear,
                                on = ['location_id','weeklastyear'],how = 'left')
dfsales_12plus.fillna(0,inplace = True)

dfsales_12plus['Store_Category'] = np.where(dfsales_12plus['open_wk']>=dfsales_12plus['weeklastyear'],'New',
                                         np.where((dfsales_12plus['gross_transaction_cnt_ly']==0)&(dfsales_12plus['class_gross_sales_amt_ly']==0),
                                         'Converted',
                                         np.where((dfsales_12plus['gross_transaction_cnt']==0)&(dfsales_12plus['class_gross_sales_amt']==0),
                                        'Converted','Complete')))
dfsales_12plus['yoysales'] = dfsales_12plus['class_gross_sales_amt']/dfsales_12plus['class_gross_sales_amt_ly'] - 1
dfsales_12plus['yoytrans'] = dfsales_12plus['gross_transaction_cnt']/dfsales_12plus['gross_transaction_cnt_ly'] - 1
dfsales_12plus = dfsales_12plus[(dfsales_12plus['Store_Category']!='Complete')|\
                                            ((dfsales_12plus['Store_Category']=='Complete')&\
                                            (abs(dfsales_12plus['yoysales'])<=0.2)|\
                                            (abs(dfsales_12plus['yoytrans'])<=0.2))]#&\
dfsales_12plus = dfsales_12plus[['location_id','week_end_dt']]

In [49]:
dfsales_rankingall = pd.DataFrame()
for i in range(52):
    cweekdate = dfweeklist2['week_end_dt'][i]
    recentweek_last = dfweeklist2['week_end_dt_8w'][i]
    dfsales_ranking = dfsales_total[dfsales_total['week_end_dt']>recentweek_last]
    dfsales_ranking = dfsales_ranking[dfsales_ranking['week_end_dt']<=cweekdate]
    dfsales_ranking = pd.merge(dfsales_ranking,dfsales_12plus,
                           on = ['location_id', 'week_end_dt'])
    dfsales_ranking = pd.merge(dfsales_ranking,
                               dftraffic[['location_id', 'week_end_dt', 'fiscal_week_nbr','traffic_week']],
                               on =['location_id', 'week_end_dt', 'fiscal_week_nbr'],how = 'left')
    dfsales_ranking.fillna(0,inplace = True)
    dfsales_ranking = dfsales_ranking[['location_id','class_gross_sales_amt','traffic_week']].groupby('location_id').sum()
    dfsales_ranking['Rev/Traffic'] = dfsales_ranking['class_gross_sales_amt']/dfsales_ranking['traffic_week']
    dfsales_ranking.reset_index(inplace = True)
    dfsales_ranking = dfsales_ranking.sort_values('class_gross_sales_amt',ascending = False)
    dfsales_ranking.reset_index(drop = True,inplace = True)
    dfsales_ranking.reset_index(inplace = True)
    dfsales_ranking = dfsales_ranking.rename(columns = {'index':'rev_index'})
    dfsales_ranking = dfsales_ranking.replace(np.inf, 0)
    
    dfsales_ranking = dfsales_ranking.sort_values('Rev/Traffic',ascending = False)
    dfsales_ranking.reset_index(drop = True,inplace = True)
    dfsales_ranking.reset_index(inplace = True)
    dfsales_ranking = dfsales_ranking.rename(columns = {'index':'traffi_index'})
    
    cols = len(dfsales_ranking.index)
    dfsales_ranking['Store_Revenue_Rank'] = np.where(dfsales_ranking['rev_index']/cols <=0.2,'H',
                                                    np.where(dfsales_ranking['rev_index']/cols <=0.8,'M','L'))
    dfsales_ranking['Store_Revenue/Traffic_Rank'] = np.where(dfsales_ranking['traffi_index']/cols <=0.2,'H',
                                                    np.where(dfsales_ranking['traffi_index']/cols <=0.8,'M','L'))
    dfsales_ranking['week_end_dt'] = cweekdate
    dfsales_rankingall = dfsales_rankingall.append(dfsales_ranking,ignore_index = False)

In [50]:
dfsales_total_recent.fillna(0,inplace = True)
dfsales_total_recent.reset_index(drop = True,inplace = True)
dfsales_total_recent = pd.merge(dfsales_total_recent,
                                dfsales_rankingall[['location_id','week_end_dt','Store_Revenue_Rank','Store_Revenue/Traffic_Rank']],
                                on = ['location_id','week_end_dt'],how = 'left')
dfsales_total_recent['Store_Revenue_Rank'].fillna('NA',inplace = True)
dfsales_total_recent['Store_Revenue/Traffic_Rank'].fillna('NA',inplace = True)

KeyError: 'Store_Revenue_Rank'

In [ ]:
dfsales_total_recent['AOV'] = dfsales_total_recent['class_gross_sales_amt']/dfsales_total_recent['gross_transaction_cnt']
dfsales_total_recent['AOV_ly'] = dfsales_total_recent['class_gross_sales_amt_ly']/dfsales_total_recent['gross_transaction_cnt_ly']
dfsales_total_recent['Trans/Traffic'] = dfsales_total_recent['gross_transaction_cnt']/dfsales_total_recent['traffic_week']
dfsales_total_recent['Trans/Traffic_ly'] = dfsales_total_recent['gross_transaction_cnt_ly']/dfsales_total_recent['traffic_week_ly']

In [ ]:
dfsales_total_recent['Store_Category'].unique()

In [51]:
df_complete = dfsales_total_recent[dfsales_total_recent['Store_Category']=='Complete']
df_complete.reset_index(drop = True, inplace = True)
metricslist = ['class_gross_sales_amt','gross_transaction_cnt','AOV','traffic_week',
              'Trans/Traffic', 'traffic_day_1', 'traffic_day_2', 'traffic_day_3',
              'traffic_day_4', 'traffic_day_5', 'traffic_day_6', 'traffic_day_7','on_hand']
columnheader = ['location_id', 'week_end_dt', 'fiscal_week_nbr', 'Store_Revenue_Rank', 'Store_Revenue/Traffic_Rank']
for i in metricslist:
    a = i+'_ly'
    b = i+ 'YoYDiff'
    columnheader = columnheader + [i,a,b]
    df_complete[b] = df_complete[i]/df_complete[a] - 1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


KeyError: 'AOV'

In [53]:
del dfstore['open_dtwd']
del dfstore['open_wk']

In [54]:
dma = pd.read_csv('/home/jubauser1/BiglotsCode/OtherInput/zipdmamapping.csv',dtype = 'str')

In [55]:
dfstore['zip_cd'] = dfstore['zip_cd'].str[0:5]
dfstore = pd.merge(dfstore,dma,on = 'zip_cd',how = 'left')

In [56]:
df_complete = df_complete[columnheader]
df_complete = pd.merge(df_complete,dfstore,on='location_id',how = 'left')
df_complete = df_complete.sort_values(['location_id','week_end_dt'],ascending = [1,0])


In [57]:
df_new = dfsales_total_recent[dfsales_total_recent['Store_Category']=='New']
df_new = df_new[['location_id', 'week_end_dt', 'fiscal_week_nbr',
                 'Store_Revenue_Rank', 'Store_Revenue/Traffic_Rank',
                 'class_gross_sales_amt','gross_transaction_cnt','AOV','traffic_week',
                 'Trans/Traffic','on_hand','Store_Category']]
df_new = pd.merge(df_new,dfstore,on='location_id',how = 'left')
df_new = df_new.sort_values(['location_id','week_end_dt'],ascending = [1,0])
del df_new['Store_Category']
df_new.to_csv(outputpath + 'output2_new.csv',index = False)

In [58]:
dfsales_total_recent['Store_Category'].unique()

array(['Complete', 'Converted', 'New'], dtype=object)

In [59]:
df_converted = dfsales_total_recent[dfsales_total_recent['Store_Category']=='Converted']
del df_converted['fiscal_week_nbr']
df_converted['week_end_dt'] = np.where(df_converted['week_end_dt']=='1970-01-01',
                                       df_converted['weeklastyear'] + pd.DateOffset(364),
                                       df_converted['week_end_dt'])
df_converted['weeklastyear'] = np.where(df_converted['weeklastyear']=='1970-01-01',
                                       df_converted['week_end_dt'] + pd.DateOffset(-364),
                                       df_converted['weeklastyear'])

df_converted = pd.merge(df_converted,dfweeklist[['week_end_dt','fiscal_week_nbr']],
                       on = 'week_end_dt',how='left')
df_converted = df_converted[['location_id', 'week_end_dt', 'fiscal_week_nbr',
                 'Store_Revenue_Rank', 'Store_Revenue/Traffic_Rank',
                 'class_gross_sales_amt','gross_transaction_cnt','AOV','traffic_week',
                 'Trans/Traffic','on_hand','Store_Category',
                 'weeklastyear','gross_transaction_cnt_ly','class_gross_sales_amt_ly']]
df_converted = pd.merge(df_converted,dfstore,on='location_id',how = 'left')
df_converted = df_converted.sort_values(['location_id','week_end_dt'],ascending = [1,0])



del df_converted['Store_Category']
df_converted.to_csv(outputpath + 'output3_converted.csv',index = False)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
df_tradearea = pd.read_csv('/home/jubauser1/BiglotsCode/OtherInput/revised_trade_area_5_mile_using_zip_center.csv',dtype = 'str')

df_tradearea_all = pd.DataFrame()
for i in (range(len(df_tradearea.index))):
    a = df_tradearea['stores'][i]
    b = df_tradearea['trade_area_code'][i]
    a = a.replace('[','')
    a = a.replace(']','')
    for c in range(a.count(',')+1):
        df_tradearea_all = df_tradearea_all.append({'location_id': (a.split(',')[c]), 'trade_area_code': b}, ignore_index=True)
df_tradearea_all['location_id'] = df_tradearea_all['location_id'].str.strip()

In [61]:
df_tradearea_all.to_csv('/home/jubauser1/BiglotsCode/OtherInput/talist.csv',index = False)

In [62]:
df_complete = pd.merge(df_complete,df_tradearea_all,on ='location_id',how = 'left')
df_complete['trade_area_code'].fillna('NA',inplace = True)
df_complete.to_csv(outputpath + 'output1.csv',index = False)

In [63]:
df_complete.to_csv(outputpath + 'output1.csv',index = False)

In [64]:
df_tadata = df_complete.groupby(['week_end_dt', 'fiscal_week_nbr','trade_area_code']).sum()
df_tadata.reset_index(inplace = True)

df_tadata['AOV'] = df_tadata['class_gross_sales_amt']/df_tadata['gross_transaction_cnt']
df_tadata['AOV_ly'] = df_tadata['class_gross_sales_amt_ly']/df_tadata['gross_transaction_cnt_ly']
df_tadata['Trans/Traffic'] = df_tadata['gross_transaction_cnt']/df_tadata['traffic_week']
df_tadata['Trans/Traffic_ly'] = df_tadata['gross_transaction_cnt_ly']/df_tadata['traffic_week_ly']

metricslist = ['class_gross_sales_amt','gross_transaction_cnt','AOV','traffic_week',
              'Trans/Traffic', 'traffic_day_1', 'traffic_day_2', 'traffic_day_3',
              'traffic_day_4', 'traffic_day_5', 'traffic_day_6', 'traffic_day_7','on_hand']
for i in metricslist:
    a = i+'_ly'
    b = i+ 'YoYDiff'
    df_tadata[b] = df_tadata[i]/df_tadata[a] - 1

In [65]:
df_taclass1 = df_complete[['location_id','trade_area_code','week_end_dt','Store_Revenue_Rank',
                           'Store_Revenue/Traffic_Rank']]
df_taclass1.reset_index(drop = True, inplace = True)
df_taclass1['Number_of_HMStores'] = np.where(df_taclass1['Store_Revenue_Rank']!='L',1,0)
df_taclass1['Number_of_HMStores_RevTrafRank'] = np.where(df_taclass1['Store_Revenue/Traffic_Rank']!='L',1,0)
df_taclass1['Number of Stores'] = 1
df_taclass1 = df_taclass1.groupby(['trade_area_code','week_end_dt']).sum()
df_taclass1.reset_index(inplace = True)

df_taclass2 = df_complete[['trade_area_code','week_end_dt','zip_cd']].drop_duplicates()
df_taclass2 = df_taclass2.groupby(['trade_area_code','week_end_dt']).count()
df_taclass2.columns = ['NumberofZipcodes']
df_taclass2.reset_index(inplace = True)

df_taclass3 = df_complete[['trade_area_code','state_nm']].drop_duplicates(['trade_area_code'])

df_tadata = pd.merge(df_tadata,df_taclass1,on =['trade_area_code','week_end_dt'])
df_tadata = pd.merge(df_tadata,df_taclass2,on =['trade_area_code','week_end_dt'])
df_tadata = pd.merge(df_tadata,df_taclass3,on =['trade_area_code'])

df_tadata = df_tadata.sort_values(['trade_area_code','week_end_dt'],ascending = [1,0])
df_tadata.to_csv(outputpath + 'output4.csv',index = False)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [66]:
df_dmadata = df_complete.groupby(['week_end_dt', 'fiscal_week_nbr','DMA']).sum()
df_dmadata.reset_index(inplace = True)

df_dmadata['AOV'] = df_dmadata['class_gross_sales_amt']/df_dmadata['gross_transaction_cnt']
df_dmadata['AOV_ly'] = df_dmadata['class_gross_sales_amt_ly']/df_dmadata['gross_transaction_cnt_ly']
df_dmadata['Trans/Traffic'] = df_dmadata['gross_transaction_cnt']/df_dmadata['traffic_week']
df_dmadata['Trans/Traffic_ly'] = df_dmadata['gross_transaction_cnt_ly']/df_dmadata['traffic_week_ly']

metricslist = ['class_gross_sales_amt','gross_transaction_cnt','AOV','traffic_week',
              'Trans/Traffic', 'traffic_day_1', 'traffic_day_2', 'traffic_day_3',
              'traffic_day_4', 'traffic_day_5', 'traffic_day_6', 'traffic_day_7','on_hand']
for i in metricslist:
    a = i+'_ly'
    b = i+ 'YoYDiff'
    df_dmadata[b] = df_dmadata[i]/df_dmadata[a] - 1

In [67]:
df_dmadata1 = df_complete[['location_id','DMA','week_end_dt','Store_Revenue_Rank',
                           'Store_Revenue/Traffic_Rank']]
df_dmadata1.reset_index(drop = True, inplace = True)
df_dmadata1['Number_of_HMStores'] = np.where(df_dmadata1['Store_Revenue_Rank']!='L',1,0)
df_dmadata1['Number_of_HMStores_RevTrafRank'] = np.where(df_dmadata1['Store_Revenue/Traffic_Rank']!='L',1,0)
df_dmadata1['Number of Stores'] = 1
df_dmadata1 = df_dmadata1.groupby(['DMA','week_end_dt']).sum()
df_dmadata1.reset_index(inplace = True)

df_dmadata2 = df_complete[['DMA','week_end_dt','zip_cd']].drop_duplicates()
df_dmadata2 = df_dmadata2.groupby(['DMA','week_end_dt']).count()
df_dmadata2.columns = ['NumberofZipcodes']
df_dmadata2.reset_index(inplace = True)

df_dmadata4 = df_complete[['DMA','week_end_dt','trade_area_code']].drop_duplicates()
df_dmadata4 = df_dmadata4.groupby(['DMA','week_end_dt']).count()
df_dmadata4.columns = ['NumberofTAs']
df_dmadata4.reset_index(inplace = True)

df_dmadata3 = df_complete[['DMA','state_nm']].drop_duplicates(['DMA'])

df_dmadata = pd.merge(df_dmadata,df_dmadata1,on =['DMA','week_end_dt'])
df_dmadata = pd.merge(df_dmadata,df_dmadata2,on =['DMA','week_end_dt'])
df_dmadata = pd.merge(df_dmadata,df_dmadata4,on =['DMA','week_end_dt'])
df_dmadata = pd.merge(df_dmadata,df_dmadata3,on =['DMA'])

df_dmadata = df_dmadata.sort_values(['DMA','week_end_dt'],ascending = [1,0])
df_dmadata.to_csv(outputpath + 'output5.csv',index = False)
del df_dmadata1,df_dmadata2,df_dmadata3,df_dmadata4

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [68]:
df_storeweeklist = pd.merge(df_complete[['location_id', 'week_end_dt']],
                            dfweeklist[['week_end_dt','weeklastyear']],on ='week_end_dt')

In [69]:
dfsales_class1 = pd.merge(dfsales[['location_id', 'week_end_dt','class_code_id','class_gross_sales_amt']],
                          df_storeweeklist,
                          on = ['location_id', 'week_end_dt'])
dfsales_class2 = pd.merge(dfsales[['location_id', 'week_end_dt', 'class_code_id','class_gross_sales_amt']],
                          df_storeweeklist,
                          left_on = ['location_id', 'week_end_dt'],
                          right_on = ['location_id', 'weeklastyear'])

del dfsales_class2['week_end_dt_x']
dfsales_class2 = dfsales_class2.rename(columns = {'week_end_dt_y':'week_end_dt'})

dfsales_class1 = pd.merge(dfsales_class1,dfsales_class2,
                          on =['location_id','week_end_dt','class_code_id'],how='outer')
dfsales_class1.fillna(0,inplace = True)
del dfsales_class2
del dfsales_class1['weeklastyear_x']
del dfsales_class1['weeklastyear_y']
dfsales_class1 = dfsales_class1.rename(columns = {'class_gross_sales_amt_x':'class_gross_sales_amt'})
dfsales_class1 = dfsales_class1.rename(columns = {'class_gross_sales_amt_y':'class_gross_sales_amt_ly'})
dfsales_class1['class_gross_sales_amt_yoy'] = dfsales_class1['class_gross_sales_amt']/dfsales_class1['class_gross_sales_amt_ly'] -1

In [70]:
dfinventory_class1 = pd.merge(dfinventory[['location_id', 'week_end_dt', 'class_code_id','on_hand']], 
                              df_storeweeklist,
                          on = ['location_id', 'week_end_dt'])
dfinventory_class2 = pd.merge(dfinventory[['location_id', 'week_end_dt', 'class_code_id','on_hand']],
                          df_storeweeklist,
                          left_on = ['location_id', 'week_end_dt'],
                          right_on = ['location_id', 'weeklastyear'])

del dfinventory_class2['week_end_dt_x']
dfinventory_class2 = dfinventory_class2.rename(columns = {'week_end_dt_y':'week_end_dt'})

dfinventory_class1 = pd.merge(dfinventory_class1,dfinventory_class2,
                          on =['location_id','week_end_dt','class_code_id'],how='outer')
dfinventory_class1.fillna(0,inplace = True)
del dfinventory_class2
del dfinventory_class1['weeklastyear_x']
del dfinventory_class1['weeklastyear_y']
dfinventory_class1 = dfinventory_class1.rename(columns = {'on_hand_x':'on_hand'})
dfinventory_class1 = dfinventory_class1.rename(columns = {'on_hand_y':'on_hand_ly'})
dfinventory_class1['on_hand_yoy'] = dfinventory_class1['on_hand']/dfinventory_class1['on_hand_ly'] -1

In [71]:
df_tadetail = pd.merge(dfsales_class1,dfinventory_class1,
                      on=['location_id','week_end_dt','class_code_id'],how='outer')
df_tadetail = pd.merge(df_tadetail,df_tradearea_all,on ='location_id',how = 'left')
df_tadetail['trade_area_code'].fillna('NA',inplace = True)
df_tadetail.fillna(0,inplace = True)

df_tadetail = df_tadetail.groupby(['trade_area_code','week_end_dt','class_code_id']).sum()
df_tadetail['class_gross_sales_amt_yoy'] = df_tadetail['class_gross_sales_amt']/df_tadetail['class_gross_sales_amt_ly'] -1
df_tadetail['on_hand_yoy'] = df_tadetail['on_hand']/df_tadetail['on_hand_ly'] -1
df_tadetail.reset_index(inplace = True)

df_tadetail = pd.merge(df_tadetail,df_taclass1,on =['trade_area_code','week_end_dt'])
df_tadetail = pd.merge(df_tadetail,df_taclass2,on =['trade_area_code','week_end_dt'])
df_tadetail = pd.merge(df_tadetail,df_taclass3,on =['trade_area_code'])
df_tadetail = pd.merge(dfweeklist[['week_end_dt','fiscal_week_nbr']],df_tadetail,
                      on ='week_end_dt')
df_tadetail.to_csv(outputpath + 'output4_classdetail.csv',index = False)

In [72]:
writer = pd.ExcelWriter(outputpath+'BigLots_Weekly_Data_'+recentweek+'.xlsx',
                            #engine='xlsxwriter',
                            datetime_format='yyyy-mm-dd',
                            date_format='yyyy-mm-dd')
test = dfsales_total[['location_id', 'week_end_dt', 'fiscal_week_nbr', 'gross_transaction_cnt']]
test = test.sort_values(['location_id', 'week_end_dt'])
test.to_excel(writer,'Transactions', index=False)
test = dfsales_total[['location_id', 'week_end_dt', 'fiscal_week_nbr', 'class_gross_sales_amt']]
test = test.sort_values(['location_id', 'week_end_dt'])
test.to_excel(writer,'Revenue', index=False)
dftraffic = dftraffic[~dftraffic['location_id'].isin(closed_onlinestorelist)]
dftraffic = dftraffic.sort_values(['location_id', 'week_end_dt'])
dfinventory_total = dfinventory_total[~dfinventory_total['location_id'].isin(closed_onlinestorelist)]
dfinventory_total = dfinventory_total.sort_values(['location_id', 'week_end_dt'])
dftraffic.to_excel(writer,'Traffic', index=False)
dfinventory_total.to_excel(writer,'Inventory', index=False)

In [73]:
writer.save()

In [74]:
df_complete_week = df_complete[df_complete['week_end_dt']==recentweek_date]
df_complete_week.to_csv(outputpath + 'output1_' + recentweek + '.csv',index = False)

df_tadetail_week = df_tadetail[df_tadetail['week_end_dt']==recentweek_date]
df_tadetail_week.to_csv(outputpath + 'output4_classdetail_' + recentweek + '.csv',index = False)

df_dmadata_week = df_dmadata[df_dmadata['week_end_dt']==recentweek_date]
df_dmadata_week.to_csv(outputpath + 'output5_' + recentweek + '.csv',index = False)

df_tadata_week = df_tadata[df_tadata['week_end_dt']==recentweek_date]
df_tadata_week.to_csv(outputpath + 'output4_' + recentweek + '.csv',index = False)

df_converted_week = df_converted[df_converted['week_end_dt']==recentweek_date]
df_converted_week.to_csv(outputpath + 'output3_converted_' + recentweek + '.csv',index = False)

df_new_week = df_new[df_new['week_end_dt']==recentweek_date]
df_new_week.to_csv(outputpath + 'output2_new_' + recentweek + '.csv',index = False)

In [75]:
df_complete_week = df_complete_week[['location_id', 'week_end_dt', 'fiscal_week_nbr', 
                                     'location_desc', 'open_dt',
                   'address_line_1', 'address_line_2', 'city_nm', 'state_nm', 'zip_cd',
                   'longitude_meas', 'latitude_meas', 'DMA', 'trade_area_code',
                   'class_gross_sales_amt',
                   'class_gross_sales_amt_ly', 'class_gross_sales_amtYoYDiff',
                   'gross_transaction_cnt', 'gross_transaction_cnt_ly','gross_transaction_cntYoYDiff',
                   'Trans/Traffic', 'Trans/Traffic_ly', 'Trans/TrafficYoYDiff',
                   'traffic_week', 'traffic_week_ly', 'traffic_weekYoYDiff']]

In [76]:
df_complete_week1 = df_complete_week.sort_values(['gross_transaction_cntYoYDiff'],ascending = True)
df_complete_week1 = df_complete_week1[df_complete_week1['gross_transaction_cntYoYDiff']<=-0.1]

df_complete_week2 = df_complete_week.sort_values(['Trans/TrafficYoYDiff'],ascending = True)
df_complete_week2 = df_complete_week2[df_complete_week2['Trans/TrafficYoYDiff']<=-0.1]

df_complete_week3 = df_complete_week.sort_values(['traffic_weekYoYDiff'],ascending = True)
df_complete_week3 = df_complete_week3[df_complete_week3['traffic_weekYoYDiff']<=-0.1]

In [77]:
import xlsxwriter

writer = pd.ExcelWriter(outputpath+'Output1Tracker_'+recentweek+'.xlsx',
                            engine='xlsxwriter',
                            datetime_format='yyyy-mm-dd',
                            date_format='yyyy-mm-dd')

workbook  = writer.book

format1 = workbook.add_format({'num_format': '#,##0'})
format2 = workbook.add_format({'text_wrap' : True})
format3 = workbook.add_format({'num_format': '0.0%'})
format4 = workbook.add_format({'num_format': '0.0%',
                               'bg_color': 'FF9999'})

df_complete_week1.to_excel(writer,'TransactionTracker', index=False)
worksheet = writer.sheets['TransactionTracker']
worksheet.set_row(0,None, format2)
worksheet.set_column('B:B', 12, None)
worksheet.set_column('E:E', 12, None)
worksheet.set_column('F:F', 16, None)
worksheet.set_column('O:Y', None, format1)
worksheet.set_column('Q:Q', None, format3)
worksheet.set_column('T:T', None, format4)
worksheet.set_column('W:W', None, format3)
worksheet.set_column('Z:Z', None, format3)
worksheet.set_column('U:V', None, format3)

df_complete_week2.to_excel(writer,'ConversionTracker', index=False)
worksheet = writer.sheets['ConversionTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('E:E', 12, None)
worksheet.set_column('F:F', 16, None)
worksheet.set_column('O:Y', None, format1)
worksheet.set_column('Q:Q', None, format3)
worksheet.set_column('T:T', None, format3)
worksheet.set_column('W:W', None, format4)
worksheet.set_column('Z:Z', None, format3)
worksheet.set_column('U:V', None, format3)

df_complete_week3.to_excel(writer,'TrafficTracker', index=False)
worksheet = writer.sheets['TrafficTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('E:E', 12, None)
worksheet.set_column('F:F', 16, None)
worksheet.set_column('O:Y', None, format1)
worksheet.set_column('Q:Q', None, format3)
worksheet.set_column('T:T', None, format3)
worksheet.set_column('W:W', None, format3)
worksheet.set_column('Z:Z', None, format4)
worksheet.set_column('U:V', None, format3)

writer.save()

In [78]:
df_complete_week = df_tadata_week[['trade_area_code', 'week_end_dt', 'fiscal_week_nbr', 
                                     'Number_of_HMStores', 'Number_of_HMStores_RevTrafRank',
                   'Number of Stores', 'NumberofZipcodes', 'state_nm',
                   'class_gross_sales_amt',
                   'class_gross_sales_amt_ly', 'class_gross_sales_amtYoYDiff',
                   'gross_transaction_cnt', 'gross_transaction_cnt_ly','gross_transaction_cntYoYDiff',
                   'Trans/Traffic', 'Trans/Traffic_ly', 'Trans/TrafficYoYDiff',
                   'traffic_week', 'traffic_week_ly', 'traffic_weekYoYDiff']]

df_complete_week1 = df_complete_week.sort_values(['gross_transaction_cntYoYDiff'],ascending = True)
df_complete_week1 = df_complete_week1[df_complete_week1['gross_transaction_cntYoYDiff']<=-0.1]

df_complete_week2 = df_complete_week.sort_values(['Trans/TrafficYoYDiff'],ascending = True)
df_complete_week2 = df_complete_week2[df_complete_week2['Trans/TrafficYoYDiff']<=-0.1]

df_complete_week3 = df_complete_week.sort_values(['traffic_weekYoYDiff'],ascending = True)
df_complete_week3 = df_complete_week3[df_complete_week3['traffic_weekYoYDiff']<=-0.1]

In [79]:
import xlsxwriter

writer = pd.ExcelWriter(outputpath+'Output4Tracker_'+recentweek+'.xlsx',
                            engine='xlsxwriter',
                            datetime_format='yyyy-mm-dd',
                            date_format='yyyy-mm-dd')

workbook  = writer.book

format1 = workbook.add_format({'num_format': '#,##0'})
format2 = workbook.add_format({'text_wrap' : True})
format3 = workbook.add_format({'num_format': '0.0%'})
format4 = workbook.add_format({'num_format': '0.0%',
                               'bg_color': 'FF9999'})

df_complete_week1.to_excel(writer,'TransactionTracker', index=False)
worksheet = writer.sheets['TransactionTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('I:S', None, format1)
worksheet.set_column('K:K', None, format3)
worksheet.set_column('N:N', None, format4)
worksheet.set_column('Q:Q', None, format3)
worksheet.set_column('T:T', None, format3)
worksheet.set_column('O:P', None, format3)

df_complete_week2.to_excel(writer,'ConversionTracker', index=False)
worksheet = writer.sheets['ConversionTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('I:S', None, format1)
worksheet.set_column('K:K', None, format3)
worksheet.set_column('N:N', None, format3)
worksheet.set_column('Q:Q', None, format4)
worksheet.set_column('T:T', None, format3)
worksheet.set_column('O:P', None, format3)

df_complete_week3.to_excel(writer,'TrafficTracker', index=False)
worksheet = writer.sheets['TrafficTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('I:S', None, format1)
worksheet.set_column('K:K', None, format3)
worksheet.set_column('N:N', None, format3)
worksheet.set_column('Q:Q', None, format3)
worksheet.set_column('T:T', None, format4)
worksheet.set_column('O:P', None, format3)

writer.save()

In [80]:
df_complete_week = df_dmadata_week[['DMA', 'week_end_dt', 'fiscal_week_nbr', 
                                     'Number_of_HMStores',
                     'Number_of_HMStores_RevTrafRank', 'Number of Stores',
                     'NumberofZipcodes', 'NumberofTAs', 'state_nm',
                   'class_gross_sales_amt',
                   'class_gross_sales_amt_ly', 'class_gross_sales_amtYoYDiff',
                   'gross_transaction_cnt', 'gross_transaction_cnt_ly','gross_transaction_cntYoYDiff',
                   'Trans/Traffic', 'Trans/Traffic_ly', 'Trans/TrafficYoYDiff',
                   'traffic_week', 'traffic_week_ly', 'traffic_weekYoYDiff']]

df_complete_week1 = df_complete_week.sort_values(['gross_transaction_cntYoYDiff'],ascending = True)
df_complete_week1 = df_complete_week1[df_complete_week1['gross_transaction_cntYoYDiff']<=-0.1]

df_complete_week2 = df_complete_week.sort_values(['Trans/TrafficYoYDiff'],ascending = True)
df_complete_week2 = df_complete_week2[df_complete_week2['Trans/TrafficYoYDiff']<=-0.1]

df_complete_week3 = df_complete_week.sort_values(['traffic_weekYoYDiff'],ascending = True)
df_complete_week3 = df_complete_week3[df_complete_week3['traffic_weekYoYDiff']<=-0.1]

In [81]:
writer = pd.ExcelWriter(outputpath+'Output5Tracker_'+recentweek+'.xlsx',
                            engine='xlsxwriter',
                            datetime_format='yyyy-mm-dd',
                            date_format='yyyy-mm-dd')

workbook  = writer.book

format1 = workbook.add_format({'num_format': '#,##0'})
format2 = workbook.add_format({'text_wrap' : True})
format3 = workbook.add_format({'num_format': '0.0%'})
format4 = workbook.add_format({'num_format': '0.0%',
                               'bg_color': 'FF9999'})

df_complete_week1.to_excel(writer,'TransactionTracker', index=False)
worksheet = writer.sheets['TransactionTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('J:T', None, format1)
worksheet.set_column('L:L', None, format3)
worksheet.set_column('O:O', None, format4)
worksheet.set_column('R:R', None, format3)
worksheet.set_column('U:U', None, format3)
worksheet.set_column('P:Q', None, format3)

df_complete_week2.to_excel(writer,'ConversionTracker', index=False)
worksheet = writer.sheets['ConversionTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('J:T', None, format1)
worksheet.set_column('L:L', None, format3)
worksheet.set_column('O:O', None, format3)
worksheet.set_column('R:R', None, format4)
worksheet.set_column('U:U', None, format3)
worksheet.set_column('P:Q', None, format3)

df_complete_week3.to_excel(writer,'TrafficTracker', index=False)
worksheet = writer.sheets['TrafficTracker']
worksheet.set_column('B:B', 12, None)
worksheet.set_column('J:T', None, format1)
worksheet.set_column('L:L', None, format3)
worksheet.set_column('O:O', None, format3)
worksheet.set_column('R:R', None, format3)
worksheet.set_column('U:U', None, format4)
worksheet.set_column('P:Q', None, format3)

writer.save()